# Evaluation

In [ ]:
import pandas as pd
import pickle
from utils import preprocessing, plotting
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score
from scipy.stats import binomtest

In [ ]:
data = pd.read_parquet('data/test.parquet')


In [ ]:
# copy from lets go - TODO: consider moving this to utils.preprocessing
y_test = ...
X_test = ...


In [ ]:
model = pickle.load(open(filename, 'rb'))


In [ ]:
y_pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred, normalize='true', labels=range(4))
crossval_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print('crossval accuracy', crossval_accuracy)
print('f1 ', f1)
print('recall ', recall)
print(conf_mat)

In [ ]:
plotting.plot_confusion_matrix(conf_mat, labels=['Centrist', 'Conservative', 'Liberal', 'Other'])

### Model Comparison

In [ ]:
result1 = binomtest(k=np.sum(y_pred_1==y_test), n=len(y_test))
result2 = ...
accuracies = [acc1, acc2]
error = [acc - result.proportion_ci()[0] for acc, result in zip(accuracies,[result1, result2])]
print(error)
print(accuracies)
sns.barplot(data=[[acc1], [acc2]])
plt.errorbar([0, 1], accuracies, yerr=error, fmt='o')